In [7]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
import pandas
import threading
import time

In [8]:
class IBapi(EWrapper, EClient):
    
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []
        self.contract_details = {}
        
    def error(self, reqId, errorCode, errorString):
        print ('Error: ', reqId, errorCode, ' ', errorString)

    def historicalData(self,reqId, bar):

        print (bar)
        self.data.append([bar.date, bar.close, bar.volume, sym_dict[reqId]])
        
    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)
        
    def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)

    def openOrder(self, orderId, contract, order, orderState):
        print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)
    
    def execDetails(self, reqId, contract, execution):
        print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares)
    
    def contractDetails(self, reqId: int, contractDetails):
        self.contract_details[reqId] = contractDetails

    def get_contract_details(self, reqId: int, contract):
        self.contract_details[reqId] = None
        self.reqContractDetails(reqId, contract)
    
    def cancel_Order(self, orderId):
        self.cancelOrder(orderId)
        print('Order'+ " " + f"{orderId}" + " " + 'canceld')

In [3]:
def run_loop():
    app.run()

def makeContract(symbol):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    return contract

In [11]:
app = IBapi()
app.connect('127.0.0.1', 4001, 0)
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()


symbols = ['SPY','MSFT','GOOG','AAPL']

reqId = 1
sym_dict = {}
for sym in symbols:
    contract = Contract()
    contract.symbol = str(sym) 
    sym_dict[reqId] = sym
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    app.reqHistoricalData(reqId, contract, "", "1 D", "10 mins", "ADJUSTED_LAST", 1, 2, False, [])
    reqId += 1

    time.sleep(5)
    
# don't sleep, use historicalDataEnd to know when finished
time.sleep(5)



Error:  -1 502   Couldn't connect to TWS. Confirm that "Enable ActiveX and Socket EClients" 
is enabled and connection port is the same as "Socket Port" on the 
TWS "Edit->Global Configuration...->API->Settings" menu. Live Trading ports: 
TWS: 7496; IB Gateway: 4001. Simulated Trading ports for new installations 
of version 954.1 or newer:  TWS: 7497; IB Gateway: 4002
Error:  1 504   Not connected
Error:  2 504   Not connected
Error:  3 504   Not connected
Error:  4 504   Not connected


In [5]:
df = pandas.DataFrame(app.data, columns=['DateTime', 'ADJUSTED_LAST','Volume','sym'])
df['DateTime'] = pandas.to_datetime(df['DateTime'],unit='s')

#make an index and sort
df = df.set_index(['sym','DateTime']).sort_index()


In [6]:
df

,,ADJUSTED_LAST,Volume
sym,DateTime,,


In [ ]:

df.loc[("SPY","2022")]


                     ADJUSTED_LAST  Volume
DateTime                                  
2022-09-23 13:30:00         368.98   43836
2022-09-23 13:40:00         368.52   37085
2022-09-23 13:50:00         367.79   28642
2022-09-23 14:00:00         368.06   27799
2022-09-23 14:10:00         368.11   19047
2022-09-23 14:20:00         368.74   18623
2022-09-23 14:30:00         367.79   17243
2022-09-23 14:40:00         367.73   11606
2022-09-23 14:50:00         367.47   19466
2022-09-23 15:00:00         368.15   16673
2022-09-23 15:10:00         368.29   10500
2022-09-23 15:20:00         367.75   14895
2022-09-23 15:30:00         367.44    9056
2022-09-23 15:40:00         366.62   18986
2022-09-23 15:50:00         366.82    9818
2022-09-23 16:00:00         366.06   12680
2022-09-23 16:10:00         366.16   14867
2022-09-23 16:20:00         365.12   18141
2022-09-23 16:30:00         365.40   17444
2022-09-23 16:40:00         365.84   12676
2022-09-23 16:50:00         366.63   14029
2022-09-23 

In [ ]:
df.to_csv('path')

In [10]:

def Stock_contract(symbol, secType='STK', exchange='SMART', currency='USD'):
    ''' custom function to create stock contract '''
    contract = Contract()
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency
    contract.primaryExchange = "NASDAQ"
    return contract

def submit_order(contract, direction, qty=100, ordertype='MKT', transmit=True):
    #Create order
    
    order = Order()
    order.action = direction
    order.totalQuantity = qty
    order.orderType = ordertype
    order.transmit = transmit
    
    #submit order
    app.placeOrder(app.nextorderId, contract, order)
    app.nextorderId += 1

In [5]:
# INITIALIZING THE ID OF THE ORDER IN THE IBAPI
app = IBapi()
    
app.connect("127.0.0.1", 7497, 0)

app.nextorderId = 1

api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(1)

The next valid order id is:  5
Error:  -1 2104   Market data farm connection is OK:usfarm.nj
Error:  -1 2104   Market data farm connection is OK:cashfarm
Error:  -1 2104   Market data farm connection is OK:usfarm
Error:  -1 2106   HMDS data farm connection is OK:euhmds
Error:  -1 2106   HMDS data farm connection is OK:ushmds.nj
Error:  -1 2106   HMDS data farm connection is OK:cashhmds
Error:  -1 2106   HMDS data farm connection is OK:fundfarm
Error:  -1 2106   HMDS data farm connection is OK:ushmds
Error:  -1 2158   Sec-def data farm connection is OK:secdefil


In [11]:

# PICK A STOCK TO BUY OR SELL
submit_order(contract = Stock_contract(symbol="AAPL"), direction="BUY")
#submit_order(contract = Stock_contract(symbol=""), direction="SELL")

Error:  6 10268   The 'EtradeOnly' order attribute is not supported.
